In [17]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

url= 'https://scholar.google.com/scholar'

#set headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36' 
    #english
}


params = {
    #english
    'hl': 'en',

    'start': 0,
    'q': 'circular economy waste household' 
}


In [2]:
#loop through pages for n pages
n = 1

from tqdm.notebook import tqdm

#store results
results = pd.DataFrame(columns=['title', 'link', 'author'])

for i in tqdm(range(n)):
    params['start'] = i*10
    response = requests.get(url, headers=headers, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('div', class_='gs_ri')

    #get titles
    titles = [article.find('h3', class_='gs_rt').text for article in articles]
    #get links
    links = [article.find('h3', class_='gs_rt').find('a')['href'] for article in articles]
    #get authors
    authors = [article.find('div', class_='gs_a').text for article in articles]
    
    #add to results
    results = results._append(pd.DataFrame({'title': titles, 'link': links, 'author': authors}))


  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
results.to_csv('tmp.csv')

In [10]:
import requests
from bs4 import BeautifulSoup

def scrape_abstract(url):
    """
    Fetch the abstract from the provided arXiv URL using XPath.
    
    Parameters:
    - url (str): The URL of the arXiv paper.

    Returns:
    - str: The abstract of the paper.
    """

    # YOUR CODE HERE
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        abstract = soup.find('blockquote', class_='abstract').text.strip()
    
    return abstract

In [20]:
link = 'https://www.sciencedirect.com/science/article/pii/S0921800919302976'

response = requests.get(link, headers=headers)

print(response)

<Response [403]>


In [41]:
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import requests
from urllib.parse import urlparse

main_page_url = 'https://sci-hub.ru/'
url_to_search = 'https://www.sciencedirect.com/science/article/pii/S0921800919302976'
driver = webdriver.Chrome()

try:
    # Open page
    driver.get(main_page_url)
    time.sleep(2)  # Wait for the page to load (you can decrease this time if need be)

    #doi_input = driver.find_element()
    # Find search bar -> input URL
    search_bar = driver.find_element(by=By.ID, value='request')
    search_bar.send_keys(url_to_search)
    search_bar.send_keys(Keys.RETURN)  # To submit the search query
    time.sleep(1)  # Wait for a bit again

    # Find download link
    download_button = driver.find_element(by=By.XPATH, value="//button[@onclick]")
    onclick_attribute = download_button.get_attribute('onclick')

    # Extract URL from the onclick attribute
    # Assuming the format is location.href='URL'
    pdf_url = onclick_attribute.split("'")[1]
    
    download_link = 'https://' + urlparse(main_page_url).hostname + pdf_url
  
    # Download the PDF using requests
    response = requests.get(download_link)

    with open('pdf.pdf', 'wb') as file:
        file.write(response.content)

except Exception as e:
    # Handle the exception, print an error message, etc.
    print(f"An error occurred: {str(e)}")
finally:
    driver.quit()

In [31]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests

main_page_url = 'https://annas-archive.org/'
doi = 'https://doi.org/10.1016/j.ecolecon.2019.106402'
driver = webdriver.Chrome()

try:
    # Open page
    driver.get(main_page_url)
    time.sleep(2)  # Wait for the page to load (you can decrease this time if need be)

    # Find DOI input field and submit the DOI
    doi_input = driver.find_element(by=By.NAME, value='doi')
    doi_input.send_keys(doi)
    doi_input.send_keys(Keys.RETURN)  # To submit the DOI
    time.sleep(1)  # Wait for a bit again

    # Find download link
    #download_button = driver.find_element(by=By.XPATH, value="//button[@onclick]")
    #onclick_attribute = download_button.get_attribute('onclick')
    download_link_element = driver.find_element(by=By.XPATH, value="//ul[@class='mb-4']//li/a[contains(@href, 'scimag')]")
    
    download_link_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//ul[@class='mb-4']//li/a[contains(@href, 'scimag')]"))
)
    download_link = download_link_element.get_attribute('href')
    # Extract URL from the onclick attribute
    # Assuming the format is location.href='URL'
    #pdf_url = onclick_attribute.split("'")[1]
    
    #download_link = 'https:' + pdf_url

    # Download the PDF using requests
    response = requests.get(download_link)
    with open('pdf.pdf', 'wb') as file:
        file.write(response.content)

finally:
    driver.quit()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//ul[@class='mb-4']//li/a[contains(@href, 'scimag')]"}
  (Session info: chrome=119.0.6045.160); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x011672A3+45731]
	(No symbol) [0x010F2D51]
	(No symbol) [0x00FE880D]
	(No symbol) [0x0101B940]
	(No symbol) [0x0101BE0B]
	(No symbol) [0x0104D1F2]
	(No symbol) [0x01038024]
	(No symbol) [0x0104B7A2]
	(No symbol) [0x01037DD6]
	(No symbol) [0x010131F6]
	(No symbol) [0x0101439D]
	GetHandleVerifier [0x01470716+3229462]
	GetHandleVerifier [0x014B84C8+3523784]
	GetHandleVerifier [0x014B214C+3498316]
	GetHandleVerifier [0x011F1680+611968]
	(No symbol) [0x010FCCCC]
	(No symbol) [0x010F8DF8]
	(No symbol) [0x010F8F1D]
	(No symbol) [0x010EB2C7]
	BaseThreadInitThunk [0x76417BA9+25]
	RtlInitializeExceptionChain [0x7710BD3B+10